In [1]:
%%capture

#| - Import Modules
import os
import sys

sys.path.append(
    os.path.join(
        os.environ["PROJ_fe_graph"],
        "04_lattice_matching",
        )
    )
from new_methods import (
    create_heterostructure,
    get_aligned_lattices,
    get_matching_lattices,
    )

import json
import pickle

from ase import io
from mpinterfaces.utils import slab_from_file

from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor
#__|

In [2]:
from mpinterfaces.interface import Interface
from pymatgen.core.lattice import Lattice
import numpy as np

In [3]:
#| - Script Inputs
strain_sys = "support"  # 'support' or 'overlayer'
# strain_sys = "overlayer"  # 'support' or 'overlayer'

bulk_filename = "init_support.cif"
graphene_filename = "init_graphene.cif"

separation = 3
nlayers_2d = 1
nlayers_substrate = 3

# Lattice matching algorithm parameters
max_area = 150
max_mismatch = 0.03
max_angle_diff = 0.03
r1r2_tol = 0.03

# max_area = 50
# max_mismatch = 0.01
# max_angle_diff = 0.01
# r1r2_tol = 0.01

max_return_structures=500
#__|

In [4]:
surface_cut = json.load(open("facet.json", "r"))["facet"]
bulk_structure = Structure.from_file(bulk_filename)
slab_structure = slab_from_file([0, 0, 1], graphene_filename)

In [5]:
# bulk_structure=None
# slab_structure=None
# strain_sys="overlayer"
# surface_cut=[0, 0, 1]
# separation=3
# nlayers_2d=1
# nlayers_substrate=4
# max_area=40
# max_mismatch=1
# max_angle_diff=0.1
# r1r2_tol=0.01
# max_return_structures=500

In [6]:
substrate_slab = Interface(
    bulk_structure,
    # substrate_bulk,
    hkl=surface_cut,
    min_thick=20,
    min_vac=30,
    primitive=False,
    from_ase=True,
    )

# mat2d_slab = slab_from_file([0, 0, 1], graphene_filename)
mat2d_slab = slab_structure

if strain_sys == "support":
    lower_mat = mat2d_slab
    upper_mat = substrate_slab
elif strain_sys == "overlayer":
    lower_mat = substrate_slab
    upper_mat = mat2d_slab

# mat2d_slab_aligned, substrate_slab_aligned = get_aligned_lattices(
# lower_mat_aligned, upper_mat_aligned = get_aligned_lattices(

# all_aligned_lattices = get_aligned_lattices(
#     lower_mat,
#     upper_mat,
#     max_area=max_area,
#     max_mismatch=max_mismatch,
#     max_angle_diff=max_angle_diff,
#     r1r2_tol=r1r2_tol,
#     )

In [7]:
slab_sub = lower_mat
slab_2d = upper_mat
max_area = max_area
max_mismatch = max_mismatch
max_angle_diff = max_angle_diff
r1r2_tol = r1r2_tol=r1r2_tol
max_return_structures = 500

# method - process_sys

In [8]:
def process_sys(
    slab_sub,
    slab_2d,
    uv_substrate,
    uv_mat2d,
    ):
    """
    """
    #| - process_sys
    substrate = Structure.from_sites(slab_sub)
    mat2d = Structure.from_sites(slab_2d)
    # map the intial slabs to the newly found matching lattices
    substrate_latt = Lattice(
        np.array(
            [
                uv_substrate[0][:],
                uv_substrate[1][:],
                substrate.lattice.matrix[2, :]
                ]
            )
        )

    # to avoid numerical issues with find_mapping
    mat2d_fake_c = mat2d.lattice.matrix[2, :] / np.linalg.norm(
        mat2d.lattice.matrix[2, :]) * 5.0
    mat2d_latt = Lattice(
        np.array(
            [
                uv_mat2d[0][:],
                uv_mat2d[1][:],
                mat2d_fake_c
                ]
            )
        )

    mat2d_latt_fake = Lattice(
        np.array(
            [
                mat2d.lattice.matrix[0, :],
                mat2d.lattice.matrix[1, :],
                mat2d_fake_c
                ]
            )
        )

    _, __, scell = substrate.lattice.find_mapping(substrate_latt,
                                                  ltol=0.05,
                                                  atol=1)
    scell[2] = np.array([0, 0, 1])
    substrate.make_supercell(scell)
    _, __, scell = mat2d_latt_fake.find_mapping(mat2d_latt,
                                                ltol=0.05,
                                                atol=1)
    scell[2] = np.array([0, 0, 1])
    mat2d.make_supercell(scell)
    # modify the substrate lattice so that the 2d material can be
    # grafted on top of it
    lmap = Lattice(
        np.array(
            [
                substrate.lattice.matrix[0, :],
                substrate.lattice.matrix[1, :],
                mat2d.lattice.matrix[2, :]
                ]
            )
        )

    mat2d.modify_lattice(lmap)

    return(substrate, mat2d)
    #__|


In [9]:
import copy
from operator import itemgetter

In [10]:
# %%timeit

# get the matching substrate and 2D material lattices
all_matching_lattices = get_matching_lattices(
    slab_sub,
    slab_2d,
    max_area=max_area,
    max_mismatch=max_mismatch,
    max_angle_diff=max_angle_diff,
    r1r2_tol=r1r2_tol,
    )

# COMBAK | Handle case where there are no systems
if all_matching_lattices is None:
    print("no matching u and v, trying adjusting the parameters")

print(20 * "o")
print(len(all_matching_lattices))
print(20 * "o")

if len(all_matching_lattices) > max_return_structures:
    print("lasfkjskfksadjfkjskldc")

    keep_sys_ind_list = []
    throw_away_ind_list = []
    for i_cnt, sys_i in enumerate(all_matching_lattices):

        sys_i["min_mismatch"] = min(sys_i["u_mismatch"], sys_i["v_mismatch"])
        sys_i["index"] = i_cnt

        min_ind = np.argmin([np.linalg.norm(i) for i in sys_i["uv1"]])

        ind_list = [0, 1]
        ind_list.remove(min_ind)

        uv_small = sys_i["uv1"][min_ind]
        uv_large = sys_i["uv1"][ind_list[0]]

        large_small_ratio = np.linalg.norm(uv_large) / np.linalg.norm(uv_small)
        large_small_ratio

        if large_small_ratio < 3.:
            keep_sys_ind_list.append(i_cnt)
        else:
            throw_away_ind_list.append(i_cnt)

    print("")
    print("Filtering by ratio")
    print(len(all_matching_lattices))
    all_matching_lattices = [all_matching_lattices[i] for i in keep_sys_ind_list]
    print(len(all_matching_lattices))

    # ##########################################################
    all_matching_lattices_1 = copy.deepcopy(all_matching_lattices)

    sorted_mismatch_list = sorted(
        all_matching_lattices_1,
        key=itemgetter('min_mismatch'),
        reverse=False)
    mismatch_ordered_indices = [i["index"] for i in sorted_mismatch_list]

    sorted_area_list = sorted(
        all_matching_lattices_1,
        key=itemgetter('min_area'),
        reverse=False)
    area_ordered_indices = [i["index"] for i in sorted_area_list]

    # Only interested in the Nth * factor (N=max_return_structures) systems in
    # terms of strain and min_area. We then take these two lists (of systems
    # that have low strain and area) and we find the union of them
    # The fudge factor is just a heuristic, since the number of entries
    # after taking the union will be < max_return_structures
    systems_to_keep_indices = list(set(
        mismatch_ordered_indices[0:int(max_return_structures * 1.5)]
            ) & set(
        area_ordered_indices[0:int(max_return_structures * 1.5)])
        )

    print("")
    print("tmptmptmp")
    print(len(all_matching_lattices))
    out = [all_matching_lattices[i] for i in systems_to_keep_indices]
    print(len(out))

    all_matching_lattices = out

#     return(None)

initial values:
uv1:
[array([2.46590000e+00, 0.00000000e+00, 1.50992827e-16]), array([-1.23295000e+00,  2.13553204e+00,  1.50992827e-16])]
uv2:
[array([2.48811000e+00, 0.00000000e+00, 1.52352797e-16]), array([-1.24405500e+00,  2.15476647e+00,  1.52352797e-16])]
 
rmax1, rmax2: 28, 27

searching ...
searching complete!


oooooooooooooooooooo
1817
oooooooooooooooooooo
lasfkjskfksadjfkjskldc

Filtering by ratio
1817
1307

tmptmptmp
1307
389


In [11]:
int(max_return_structures * 1.5)

750

In [12]:
# sys_i["min_mismatch"] = min(sys_i["u_mismatch"], sys_i["v_mismatch"])

In [13]:
len(all_matching_lattices)

389

In [14]:
print("la;skd;fpoiweqir;asldkjpwo2eiru;sldkjf")

la;skd;fpoiweqir;asldkjpwo2eiru;sldkjf


In [15]:
assert False

AssertionError: 

In [ ]:
# all_matching_lattices = get_matching_lattices(
#     slab_sub,
#     slab_2d,
#     max_area=max_area,
#     max_mismatch=max_mismatch,
#     max_angle_diff=max_angle_diff,
#     r1r2_tol=r1r2_tol,
#     )

In [ ]:
# %%timeit
print("Creating heterointerfaces...")
sub_mat2d_list_tmp = []
for sys_i in all_matching_lattices:
    #| - body
    uv_substrate = sys_i["uv1"]
    uv_mat2d = sys_i["uv2"]

    substrate, mat2d = process_sys(
        slab_sub,
        slab_2d,
        uv_substrate,
        uv_mat2d,
        )

    sub_mat2d_list_tmp.append(
        {
            **sys_i,
            "substrate": substrate,
            "mat2d": mat2d,
            }
        )

    #__|

print("Done creating heterointerfaces!"); print("\n")
#     return(sub_mat2d_list_tmp)

In [ ]:
# keep_sys_ind_list = []
# throw_away_ind_list = []
# for i_cnt, sys_i in enumerate(all_matching_lattices):

#     min_ind = np.argmin([np.linalg.norm(i) for i in sys_i["uv1"]])

#     ind_list = [0, 1]
#     ind_list.remove(min_ind)

#     uv_small = sys_i["uv1"][min_ind]
#     uv_large = sys_i["uv1"][ind_list[0]]

#     large_small_ratio = np.linalg.norm(uv_large) / np.linalg.norm(uv_small)
#     large_small_ratio

#     if large_small_ratio < 3.:
#         keep_sys_ind_list.append(i_cnt)
#     else:
#         throw_away_ind_list.append(i_cnt)

# print(all_matching_lattices)
# all_matching_lattices = [all_matching_lattices[i] for i in keep_sys_ind_list]
# print(all_matching_lattices)

# # ##########################################################

# all_matching_lattices_1 = copy.deepcopy(all_matching_lattices)

# sorted_mismatch_list = sorted(
#     all_matching_lattices_1,
#     key=itemgetter('min_mismatch'),
#     reverse=False)
# mismatch_ordered_indices = [i["index"] for i in sorted_area_list]

# sorted_area_list = sorted(
#     all_matching_lattices_1,
#     key=itemgetter('min_area'),
#     reverse=False)
# area_ordered_indices = [i["index"] for i in sorted_area_list]

# systems_to_keep_indices = list(set(
#     mismatch_ordered_indices[0:max_return_structures]
#         ) & set(
#     area_ordered_indices[0:max_return_structures])
#     )
# out = [all_matching_lattices[i] for i in systems_to_keep_indices]

In [ ]:
# area_data = [i["min_area"] for i in sorted_area_list]

In [ ]:
# print(len(keep_sys_ind_list))
# print(len(throw_away_ind_list))

In [ ]:
# all_matching_lattices[0]["uv1"][0]

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Fixing random state for reproducibility
# np.random.seed(19680801)


# N = 50
# x = list(range(len(area_data)))
# y = area_data
# # colors = np.random.rand(N)
# # area = (30 * np.random.rand(N))**2  # 0 to 15 point radii

# plt.scatter(x, y,
# #     s=area,
# #     c=colors,
#     alpha=0.5,
#     )
# # plt.show()

In [ ]:
# print(len(all_matching_lattices))

# list(all_matching_lattices[0].keys())

In [ ]:
# pickle.load(
#     open("heterostructures.pickle", "rb"),
#     )

In [ ]:
# min_ind = np.argmin([np.linalg.norm(i) for i in sys_i["uv1"]])

# ind_list = [0, 1]
# ind_list.remove(min_ind)

# uv_small = sys_i["uv1"][min_ind]
# uv_large = sys_i["uv1"][ind_list[0]]

# large_small_ratio = np.linalg.norm(uv_large) / np.linalg.norm(uv_small)
# large_small_ratio

# if large_small_ratio > 4:
#     tmp = 42
#     print("lskjf")